In [6]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import re
import ast
import pandas as pd
import numpy as np
import random

In [3]:
userAgents = [
    'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'
]

In [2]:
hochiminh_districts = [
    "quán ăn quận 1",
    "quán ăn quận 2",
    "quán ăn quận 3",
    "quán ăn quận 5",
    "quán ăn quận 7",
    "quán ăn quận 10",
    "quán ăn quận Bình Thạnh",
    "quán ăn quận Gò Vấp",
    "quán ăn quận Phú Nhuận",
    "quán ăn quận Tân Bình",
    "quán ăn quận Thủ Đức"
]

In [3]:
hanoi_districts = [
    "quán ăn quận Ba Đình",
    "quán ăn quận Hoàn Kiếm",
    "quán ăn quận Hai Bà Trưng",
    "quán ăn quận Đống Đa",
    "quán ăn quận Cầu Giấy",
    "quán ăn quận Thanh Xuân",
    "quán ăn quận Tây Hồ",
    "quán ăn quận Hoàng Mai",
    "quán ăn quận Long Biên",
    "quán ăn quận Hà Đông"
]

In [4]:
danang_districts = [
    "quán ăn quận Hải Châu",
    "quán ăn quận Thanh Khê",
    "quán ăn quận Sơn Trà",
    "quán ăn quận Ngũ Hành Sơn",
    "quán ăn quận Liên Chiểu",
    "quán ăn quận Cẩm Lệ"
]

In [12]:
baseDomain = 'https://www.google.com/maps?hl=vi'
driverdir = ChromeDriverManager().install()
service = Service(executable_path=driverdir)
driver = webdriver.Chrome(service=service)
driver.get(baseDomain)

## 1. URL Collection

In [13]:
def search(keyword : str):
    searchBox = driver.find_element(By.ID,"searchboxinput")   # Find search bar
    searchBox.clear()                                         # Make search bar empty ""
    searchBox.send_keys(keyword)                              # Pass keyword to search bar for searching
    searchButton = driver.find_element(By.CLASS_NAME,"mL3xi") # Find lookup button
    searchButton.click()                                      # Click on the lookup button

In [ ]:
def scroll(scrollTime : int):
    # Find the window element to scroll
    element = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]')
    while scrollTime > 0:
        # Run JavaScript code
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", element)
        time.sleep(2) # Sleep the program 2 seconds to avoid overlapping
        scrollTime -= 1

In [15]:
search('nhà hàng quán ăn quận 1')

In [114]:
scroll(20)

In [ ]:
urlList = []
for keyword in hochiminh_districts + hanoi_districts + danang_districts:
   search(keyword)
   time.sleep(2) # Wait for the process of searching
   scroll(20)
   
   # Collect all results as a list
   results = driver.find_elements(By.CLASS_NAME,"hfpxzc")
   
   for ele in results:
      # Store the url of each result
      urlList.append(ele.get_attribute('href'))

In [ ]:
saveName = 'total-diner-urls.txt'
with open(file=saveName,mode='w') as saveFile:
    for ele in set(urlList):
        saveFile.write(ele.strip() + '\n')

## 2. Review Collection

### a. URL Filter (restaurants at least 100 reviews)

In [ ]:
driverdir = ChromeDriverManager().install()
service = Service(executable_path=driverdir)
option = Options()
option.add_argument("--headless=new")


def get_digits(text : str):
    # This function will take only digits from a string 
    return int(''.join([c for c in text if c.isdigit()]))

def filter_diner_urls(url : str):
    # This function will filter the urls less than 100 reviews
    driver = webdriver.Chrome(service=service,options=option)
    driver.get(url)
    try:
        rating = get_digits(driver.find_element(By.XPATH,"//*[@class='F7nice ']").find_elements(By.TAG_NAME,'span')[-1].text)
        driver.quit()
        return url if rating >= 100 else None
    except:
        driver.quit()
        return None

In [30]:
fileName = 'raw-url/total-diner.txt'
with open(fileName,'r') as readFile:
    totalList = [ele.strip() for ele in readFile.readlines()]

In [ ]:
res = []
saveFile = 'diner-at-least-100-reviews.txt'
with open(saveFile,'w') as saveFile:
    for i,ele in enumerate(totalList):
        if i % 500 == 0:
            print(i)
        try:
            extractedData = filter_diner_urls(ele)
            if extractedData:
                saveFile.write(extractedData + '\n')
                saveFile.flush()
                res.append(extractedData)
        except:
            continue

0
500
1000
1500
2000
2500
3000
3500
4000


### b. Review Extraction

In [ ]:
def review_extractor(url : str):
    # Driver set up
    driverdir = ChromeDriverManager().install()
    service = Service(executable_path=driverdir)
    option = Options()
    option.add_argument("--headless=new")
    driver = webdriver.Chrome(service=service,options=option)
    driver.get(url)
    
    # Wait for the process of requesting url
    time.sleep(2)
    
    try:
        # Find the review tab element and click on it
        driver.find_element(By.XPATH,"//*[@class='hh2c6 ']").click()
        time.sleep(2)
    except:
        # If not exist, skip this url
        driver.quit()
        return None
    
    # Find the window of reviews and scroll down 10 times
    scroller = driver.find_element(By.XPATH,"//*[@class='m6QErb DxyBCb kA9KIf dS8AEf XiKgde ']")
    scrollTime = 10
    for i in range(scrollTime):
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scroller)
        time.sleep(1)
    
    # Collect all the reviews after scrolling
    reviewList = driver.find_elements(By.XPATH,"//*[@class='GHT2ce']")
    
    res = []
    for ele in reviewList:
        # Try to click on the "More" button to fully render the text
        try:
            ele.find_element(By.XPATH,"//*[@class='w8nwRe kyuRq']").click()
            time.sleep(1)
        except:
            pass
        
        try:
            # Get the rating of review
            rating = ele.find_element(By.CLASS_NAME,"kvMYJc").get_attribute('aria-label')
            # Get the text of review
            review = ele.find_element(By.CLASS_NAME,'MyEned').find_element(By.CLASS_NAME,"wiI7pd").text
            res.append([rating,review])
        except:
            continue
    return res

In [4]:
fileName = 'diner-at-least-100-reviews.txt'
with open(fileName,'r') as readFile:
    urlList = [ele.strip() for ele in readFile.readlines()]

In [ ]:
saveName = 'diner-reviews.txt'
resumeName = 'resume.txt'

# Set up resume mode
try:
    resumeLine = int(open(resumeName,'r').readline())
except:
    resumeLine = -1

with open(saveName,'a') as saveFile:
    for i,ele in enumerate(urlList):
        if i < resumeLine:
            continue
        with open(resumeName,'w') as resumeFile:
            resumeFile.write(str(i)) 
            resumeFile.flush()
        try:
            reviews = review_extractor(ele)
            for rev in reviews:
                saveFile.write(str(rev) + '\n')
                saveFile.flush()
        except KeyboardInterrupt:
            break
        except:
            continue